### Movie Dataset

In [67]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "imdb_top_1000.csv"
movies_df = pd.read_csv(file_path)


In [68]:
df1 = pd.read_csv("tmdb_5000_credits.csv")
df2 = pd.read_csv("movies_metadata.csv")

/var/folders/s0/399qjfv1781c67074w7lg0tc0000gn/T/ipykernel_45095/1464754924.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("movies_metadata.csv")


In [69]:
df2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [59]:
df1.columns = ['id', 'title', 'cast', 'crew']
df2 = df2.merge(df1, on='id')
df2

MergeError: Passing 'suffixes' which cause duplicate columns {'crew_x', 'cast_x'} is not allowed.

In [27]:
# Print first few rows of the dataset
movies_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [28]:
# Get general information about the dataframe
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [29]:
# Summary statistics of numerical columns
movies_df.describe()

,IMDB_Rating,Meta_score,No_of_Votes
count,1000.000000,843.000000,1.000000e+03
mean,7.949300,77.971530,2.736929e+05
std,0.275491,12.376099,3.273727e+05
min,7.600000,28.000000,2.508800e+04
25%,7.700000,70.000000,5.552625e+04
50%,7.900000,79.000000,1.385485e+05
75%,8.100000,87.000000,3.741612e+05
max,9.300000,100.000000,2.343110e+06


In [30]:
movies_df.set_index('Series_Title', inplace=True) # Change index to be Title column
movies_df.head(4)

,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Series_Title,,,,,,,,,,,,,,,
The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"


In [31]:
movies_df.tail(10)

,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Series_Title,,,,,,,,,,,,,,,
Giù la testa,https://m.media-amazon.com/images/M/MV5BYjRmY2...,1971,PG,157 min,"Drama, War, Western",7.6,A low-life bandit and an I.R.A. explosives exp...,77.0,Sergio Leone,Rod Steiger,James Coburn,Romolo Valli,Maria Monti,30144,"696,690"
Kelly's Heroes,https://m.media-amazon.com/images/M/MV5BMzAyND...,1970,GP,144 min,"Adventure, Comedy, War",7.6,A group of U.S. soldiers sneaks across enemy l...,50.0,Brian G. Hutton,Clint Eastwood,Telly Savalas,Don Rickles,Carroll O'Connor,45338,"1,378,435"
The Jungle Book,https://m.media-amazon.com/images/M/MV5BMjAwMT...,1967,U,78 min,"Animation, Adventure, Family",7.6,Bagheera the Panther and Baloo the Bear have a...,65.0,Wolfgang Reitherman,Phil Harris,Sebastian Cabot,Louis Prima,Bruce Reitherman,166409,"141,843,612"
Blowup,https://m.media-amazon.com/images/M/MV5BYTE4YW...,1966,A,111 min,"Drama, Mystery, Thriller",7.6,A fashion photographer unknowingly captures a ...,82.0,Michelangelo Antonioni,David Hemmings,Vanessa Redgrave,Sarah Miles,John Castle,56513,NaN
A Hard Day's Night,https://m.media-amazon.com/images/M/MV5BZjQyMG...,1964,U,87 min,"Comedy, Music, Musical",7.6,"Over two ""typical"" days in the life of The Bea...",96.0,Richard Lester,John Lennon,Paul McCartney,George Harrison,Ringo Starr,40351,"13,780,024"
Breakfast at Tiffany's,https://m.media-amazon.com/images/M/MV5BNGEwMT...,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
Giant,https://m.media-amazon.com/images/M/MV5BODk3Yj...,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
From Here to Eternity,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
Lifeboat,https://m.media-amazon.com/images/M/MV5BZTBmMj...,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


In [32]:
movies_df['Rank'] = range(1, len(movies_df) + 1)

I want to move the new column to the position after the Series Title (index 0)

In [33]:
# get list of column names
columns = list(movies_df.columns)
# remove the 'Rank' column from the list
columns.remove('Rank')
# insert 'Rank' column at specified index
columns.insert(0, 'Rank')
# Reorder the DataFrame Columns
movies_df = movies_df[columns]
# Check DataFrame info
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, The Shawshank Redemption to The 39 Steps
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           1000 non-null   int64  
 1   Poster_Link    1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 132.8+ KB


In [34]:
movies_df = movies_df.sort_values(by='IMDB_Rating', ascending=False)

movies_df.reset_index(inplace=True)

movies_df.set_index('Rank', inplace=True)

movies_df

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
2,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
3,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
4,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
5,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,Zombieland,https://m.media-amazon.com/images/M/MV5BMTU5MD...,2009,A,88 min,"Adventure, Comedy, Fantasy",7.6,A shy student trying to reach his family in Oh...,73.0,Ruben Fleischer,Jesse Eisenberg,Emma Stone,Woody Harrelson,Abigail Breslin,520041,"75,590,286"
912,La piel que habito,https://m.media-amazon.com/images/M/MV5BYmFmNj...,2011,R,120 min,"Drama, Horror, Thriller",7.6,"A brilliant plastic surgeon, haunted by past t...",70.0,Pedro Almodóvar,Antonio Banderas,Elena Anaya,Jan Cornet,Marisa Paredes,138959,"3,185,812"
911,Moneyball,https://m.media-amazon.com/images/M/MV5BMjAxOT...,2011,PG-13,133 min,"Biography, Drama, Sport",7.6,Oakland A's general manager Billy Beane's succ...,87.0,Bennett Miller,Brad Pitt,Robin Wright,Jonah Hill,Philip Seymour Hoffman,369529,"75,605,492"


In [35]:
movies_df.shape

(1000, 16)

In [36]:
movies_df.size

16000

In [37]:
movies_df.iloc[199] # Get info about movie rank 200

Series_Title                                               Network
Poster_Link      https://m.media-amazon.com/images/M/MV5BZGNjYj...
Released_Year                                                 1976
Certificate                                                     UA
Runtime                                                    121 min
Genre                                                        Drama
IMDB_Rating                                                    8.1
Overview         A television network cynically exploits a dera...
Meta_score                                                    83.0
Director                                              Sidney Lumet
Star1                                                 Faye Dunaway
Star2                                               William Holden
Star3                                                  Peter Finch
Star4                                                Robert Duvall
No_of_Votes                                                 14

In [38]:
movies_df.sort_index().head()

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
2,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
3,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
4,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
5,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [39]:
movies_df['Genre']

Rank
1                            Drama
2                     Crime, Drama
3             Action, Crime, Drama
4                     Crime, Drama
5                     Crime, Drama
                   ...            
913     Adventure, Comedy, Fantasy
912        Drama, Horror, Thriller
911        Biography, Drama, Sport
910       Action, Adventure, Crime
1000      Crime, Mystery, Thriller
Name: Genre, Length: 1000, dtype: object

In [40]:
movies_df['Genre'].value_counts().head(40)

Genre
Drama                           85
Drama, Romance                  37
Comedy, Drama                   35
Comedy, Drama, Romance          31
Action, Crime, Drama            30
Biography, Drama, History       28
Crime, Drama, Thriller          28
Crime, Drama, Mystery           27
Crime, Drama                    26
Animation, Adventure, Comedy    24
Action, Adventure, Sci-Fi       21
Biography, Crime, Drama         16
Comedy, Crime, Drama            15
Drama, War                      15
Drama, Thriller                 14
Action, Adventure, Drama        14
Comedy                          13
Biography, Drama                12
Animation, Action, Adventure    11
Action, Crime, Thriller         10
Action, Adventure, Comedy       10
Animation, Adventure, Family     9
Drama, Mystery, Thriller         9
Biography, Drama, Sport          9
Mystery, Thriller                9
Adventure, Comedy, Drama         8
Animation, Adventure, Drama      7
Drama, Music, Romance            7
Biography, Dra

In [41]:
movies_df[movies_df['Genre'] == 'Drama']

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
10,Fight Club,https://m.media-amazon.com/images/M/MV5BMmEzNT...,1999,A,139 min,Drama,8.8,An insomniac office worker and a devil-may-car...,66.0,David Fincher,Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier,1854740,"37,030,102"
18,One Flew Over the Cuckoo's Nest,https://m.media-amazon.com/images/M/MV5BZjA0OW...,1975,A,133 min,Drama,8.7,A criminal pleads insanity and is admitted to ...,83.0,Milos Forman,Jack Nicholson,Louise Fletcher,Michael Berryman,Peter Brocco,918088,"112,000,000"
21,Soorarai Pottru,https://m.media-amazon.com/images/M/MV5BOTc2ZT...,2020,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,NaN
41,American History X,https://m.media-amazon.com/images/M/MV5BZjA0MT...,1998,R,119 min,Drama,8.5,A former neo-nazi skinhead tries to prevent hi...,62.0,Tony Kaye,Edward Norton,Edward Furlong,Beverly D'Angelo,Jennifer Lien,1034705,"6,719,864"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,Nelyubov,https://m.media-amazon.com/images/M/MV5BNzk2Nm...,2017,R,127 min,Drama,7.6,A couple going through a divorce must team up ...,86.0,Andrey Zvyagintsev,Maryana Spivak,Aleksey Rozin,Matvey Novikov,Marina Vasileva,29765,"566,356"
935,Mysterious Skin,https://m.media-amazon.com/images/M/MV5BMTgxMj...,2004,R,105 min,Drama,7.6,A teenage hustler and a young man obsessed wit...,73.0,Gregg Araki,Brady Corbet,Joseph Gordon-Levitt,Elisabeth Shue,Chase Ellison,65939,"697,181"
934,"Synecdoche, New York",https://m.media-amazon.com/images/M/MV5BMjA0Mj...,2008,R,124 min,Drama,7.6,"A theatre director struggles with his work, an...",67.0,Charlie Kaufman,Philip Seymour Hoffman,Samantha Morton,Michelle Williams,Catherine Keener,83158,"3,081,925"


In [42]:
after_year = movies_df['Released_Year'] > '2015'
movies_df[after_year]

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
21,Soorarai Pottru,https://m.media-amazon.com/images/M/MV5BOTc2ZT...,2020,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,NaN
20,Gisaengchung,https://m.media-amazon.com/images/M/MV5BYWZjMj...,2019,A,132 min,"Comedy, Drama, Thriller",8.6,Greed and class discrimination threaten the ne...,96.0,Bong Joon Ho,Kang-ho Song,Lee Sun-kyun,Cho Yeo-jeong,Choi Woo-sik,552778,"53,367,844"
19,Hamilton,https://m.media-amazon.com/images/M/MV5BNjViNW...,2020,PG-13,160 min,"Biography, Drama, History",8.6,The real life of one of America's foremost fou...,90.0,Thomas Kail,Lin-Manuel Miranda,Phillipa Soo,Leslie Odom Jr.,Renée Elise Goldsberry,55291,NaN
34,Joker,https://m.media-amazon.com/images/M/MV5BNGVjNW...,2019,A,122 min,"Crime, Drama, Thriller",8.5,"In Gotham City, mentally troubled comedian Art...",59.0,Todd Phillips,Joaquin Phoenix,Robert De Niro,Zazie Beetz,Frances Conroy,939252,"335,451,311"
60,Avengers: Endgame,https://m.media-amazon.com/images/M/MV5BMTc5MD...,2019,UA,181 min,"Action, Adventure, Drama",8.4,After the devastating events of Avengers: Infi...,78.0,Anthony Russo,Joe Russo,Robert Downey Jr.,Chris Evans,Mark Ruffalo,809955,"858,373,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,The Florida Project,https://m.media-amazon.com/images/M/MV5BMjg4Zm...,2017,A,111 min,Drama,7.6,"Set over one summer, the film follows precocio...",92.0,Sean Baker,Brooklynn Prince,Bria Vinaite,Willem Dafoe,Christopher Rivera,95181,"5,904,366"
881,Nelyubov,https://m.media-amazon.com/images/M/MV5BNzk2Nm...,2017,R,127 min,Drama,7.6,A couple going through a divorce must team up ...,86.0,Andrey Zvyagintsev,Maryana Spivak,Aleksey Rozin,Matvey Novikov,Marina Vasileva,29765,"566,356"
880,Once Upon a Time... in Hollywood,https://m.media-amazon.com/images/M/MV5BOTg4ZT...,2019,A,161 min,"Comedy, Drama",7.6,A faded television actor and his stunt double ...,83.0,Quentin Tarantino,Leonardo DiCaprio,Brad Pitt,Margot Robbie,Emile Hirsch,551309,"142,502,728"


In [43]:
# Handling missing data
print(movies_df.isna().sum())


Series_Title       0
Poster_Link        0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64


In [44]:
genre_group = movies_df.groupby('Genre').agg({'IMDB_Rating': 'mean', 'Gross': 'sum'})
print(genre_group)

                              IMDB_Rating  \
Genre                                       
Action, Adventure                8.180000   
Action, Adventure, Biography     7.900000   
Action, Adventure, Comedy        7.910000   
Action, Adventure, Crime         7.600000   
Action, Adventure, Drama         8.150000   
...                                   ...   
Mystery, Romance, Thriller       8.300000   
Mystery, Sci-Fi, Thriller        7.800000   
Mystery, Thriller                7.977778   
Thriller                         7.800000   
Western                          8.350000   

                                                                          Gross  
Genre                                                                            
Action, Adventure             248,159,971448,139,099197,171,806206,852,43247...  
Action, Adventure, Biography                                                  0  
Action, Adventure, Comedy     1,033,895363,070,709333,176,6008,178,00157,229...  
Acti

In [45]:
# String cleaning
movies_df['Genre'] = movies_df['Genre'].str.strip().str.lower()
print(movies_df['Genre'].head())

Rank
1                   drama
2            crime, drama
3    action, crime, drama
4            crime, drama
5            crime, drama
Name: Genre, dtype: object


In [46]:
movies_df.duplicated().sum()

0

In [47]:
# correcting data types
movies_df['Gross'] = movies_df['Gross'].str.replace(',', '').astype(float)
movies_df

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0
2,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"crime, drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
3,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"action, crime, drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
4,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"crime, drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0
5,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,U,96 min,"crime, drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,Zombieland,https://m.media-amazon.com/images/M/MV5BMTU5MD...,2009,A,88 min,"adventure, comedy, fantasy",7.6,A shy student trying to reach his family in Oh...,73.0,Ruben Fleischer,Jesse Eisenberg,Emma Stone,Woody Harrelson,Abigail Breslin,520041,75590286.0
912,La piel que habito,https://m.media-amazon.com/images/M/MV5BYmFmNj...,2011,R,120 min,"drama, horror, thriller",7.6,"A brilliant plastic surgeon, haunted by past t...",70.0,Pedro Almodóvar,Antonio Banderas,Elena Anaya,Jan Cornet,Marisa Paredes,138959,3185812.0
911,Moneyball,https://m.media-amazon.com/images/M/MV5BMjAxOT...,2011,PG-13,133 min,"biography, drama, sport",7.6,Oakland A's general manager Billy Beane's succ...,87.0,Bennett Miller,Brad Pitt,Robin Wright,Jonah Hill,Philip Seymour Hoffman,369529,75605492.0


In [51]:
# Merge
movies = pd.read_csv("Highest Hollywood Grossing Movies.csv")
movies.info()
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  1000 non-null   int64 
 1   Title                       1000 non-null   object
 2   Movie Info                  1000 non-null   object
 3   Year                        1000 non-null   int64 
 4   Distributor                 999 non-null    object
 5   Budget (in $)               1000 non-null   object
 6   Domestic Opening (in $)     1000 non-null   object
 7   Domestic Sales (in $)       1000 non-null   int64 
 8   International Sales (in $)  1000 non-null   int64 
 9   World Wide Sales (in $)     1000 non-null   int64 
 10  Release Date                1000 non-null   object
 11  Genre                       1000 non-null   object
 12  Running Time                1000 non-null   object
 13  License                     943 non-null    objec